# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
city_df = pd.read_csv("city_data_export.csv")
city_df.drop(['Unnamed: 0'], axis=1)
city_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mount isa,-20.7333,139.5000,84.20,28.0,75.0,4.61,AU,1.614526e+09
1,1,zaysan,47.4667,84.8667,9.88,90.0,100.0,0.92,KZ,1.614526e+09
2,2,punta arenas,-53.1500,-70.9167,62.60,67.0,75.0,18.41,CL,1.614526e+09
3,3,mataura,-46.1927,168.8643,60.01,95.0,93.0,3.00,NZ,1.614526e+09
4,4,lebu,-37.6167,-73.6500,63.32,71.0,0.0,17.85,CL,1.614526e+09
...,...,...,...,...,...,...,...,...,...,...
525,525,baghdad,33.3406,44.4009,57.20,51.0,75.0,9.22,IQ,1.614526e+09
526,526,kasongo-lunda,-6.4667,16.8167,89.11,54.0,13.0,4.09,CD,1.614526e+09
527,527,chinsali,-10.5414,32.0816,77.25,74.0,35.0,0.94,ZM,1.614526e+09
528,528,general artigas,-26.9333,-56.2167,87.80,51.0,0.0,5.75,PY,1.614526e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
locations = city_df[['Lat','Lng']]
humidity_weights = city_df['Humidity']
max_humidity = humidity_weights.max()

fig = gmaps.figure(map_type="TERRAIN",center=(40,15),zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations,
                                 weights=humidity_weights, 
                                 dissipating=False, 
                                 max_intensity=max_humidity,
                                 point_radius=3,
                                 )

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
narrow_df = city_df.loc[(city_df['Max Temp']>=70) & 
                        (city_df['Max Temp']<= 85) & 
                        (city_df['Wind Speed']< 10) &
                        (city_df['Humidity']< 80) &
                        (city_df['Cloudiness']< 20)
                        ]
narrow_df = narrow_df.dropna(how='any')
narrow_df.reset_index(inplace=True)
narrow_df.head(10)

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,152,152,cartagena,10.3997,-75.5144,84.20,70.0,0.0,4.61,CO,1.614526e+09
1,238,238,ibiruba,-28.6275,-53.0897,79.50,58.0,0.0,9.78,BR,1.614526e+09
2,247,247,coahuayana,18.7333,-103.6833,77.00,47.0,1.0,0.76,MX,1.614526e+09
3,262,262,kununurra,-15.7667,128.7333,77.00,55.0,14.0,1.14,AU,1.614526e+09
4,320,320,tomatlan,19.9333,-105.2500,75.96,48.0,0.0,0.89,MX,1.614526e+09
5,324,324,hualmay,-11.0964,-77.6139,77.22,61.0,4.0,6.82,PE,1.614526e+09
6,333,333,jibuti,11.5000,42.5000,78.80,69.0,1.0,5.75,DJ,1.614526e+09
7,366,366,kavaratti,10.5669,72.6420,83.08,68.0,0.0,4.00,IN,1.614526e+09
8,372,372,general roca,-39.0333,-67.5833,80.60,39.0,0.0,5.75,AR,1.614526e+09
9,382,382,bilma,18.6853,12.9164,84.58,6.0,0.0,6.58,NE,1.614526e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df = []

for i in range(len(narrow_df)):
    lat = narrow_df.loc[i]['Lat']
    lng = narrow_df.loc[i]['Lng']
    
    parameters = {
        'location': f"{lat},{lng}",
        'radius': 5000,
        'type': 'hotel',
        'key': g_key
    }
    
    base_gmaps_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response  = requests.get(base_gmaps_url, params=parameters)
    response_json = response.json()
    
    try:
        hotel_df.append(response_json['results'][0]['name'])
    except:
        hotel_df.append('')
narrow_df['Hotel Name'] = hotel_df
narrow_df = narrow_df.dropna(how='any')
narrow_df.sample(10)

,index,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,152,152,cartagena,10.3997,-75.5144,84.20,70.0,0.0,4.61,CO,1.614526e+09,Cartagena
8,372,372,general roca,-39.0333,-67.5833,80.60,39.0,0.0,5.75,AR,1.614526e+09,General Roca
18,514,514,lima,-12.0432,-77.0282,78.80,65.0,0.0,9.22,PE,1.614526e+09,Lima
7,366,366,kavaratti,10.5669,72.6420,83.08,68.0,0.0,4.00,IN,1.614526e+09,Kavaratti
17,513,513,port blair,11.6667,92.7500,79.20,63.0,0.0,4.14,IN,1.614526e+09,Port Blair
16,455,455,guanica,17.9716,-66.9080,82.99,59.0,4.0,5.01,PR,1.614526e+09,Guanica
20,524,524,ishigaki,24.3448,124.1572,70.11,75.0,5.0,4.70,JP,1.614526e+09,Ishigaki
9,382,382,bilma,18.6853,12.9164,84.58,6.0,0.0,6.58,NE,1.614526e+09,Bilma
13,423,423,acapulco,16.8634,-99.8901,78.80,78.0,1.0,3.44,MX,1.614526e+09,Acapulco
14,433,433,rawson,-43.3002,-65.1023,79.79,22.0,8.0,8.25,AR,1.614526e+09,Rawson


In [23]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
#<dt>Country</dt><dd>{Country}</dd>
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrow_df.iterrows()]
hotel_locations = narrow_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
# fig = gmaps.figure(map_type="TERRAIN",center=(40,15),zoom_level=2)
markers = gmaps.marker_layer(locations)

# Add layer
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))